In [1]:
import pandas as pd
import datetime
import statsmodels.api as sm
from statsmodels.stats.weightstats import DescrStatsW
from scipy.stats import pearsonr, spearmanr
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import json
from tqdm import tqdm

pd.set_option('max_rows', 500)

Todos: 

1. improve filtering? (Not sure what stuff we should be filtering out.) Improve duplicate checks? 
2. Make sure all the column names are correct. In particular, I'm not sure if I'm using the right location grouping column, and we should be using consistent tract/block group fields. For example, is median_household_income at tract or Block group level? 


In [3]:
VALSET_PATH = '/share/pierson/nexar_data/dashcam-analysis/final_model_metrics/valset_2.csv'
BASE_CHUNKS_PATH = '/share/pierson/nexar_data/FINAL_CHUNKS_ETHNICITY_DATA/%i.csv'
N_CHUNKS = 20
COLS_TO_DEDUPLICATE_ON = ['lat', 'lng', 'timestamp'] # columns to use to check for duplicates
MIN_DATE_FOR_DEMOGRAPHIC_ANALYSIS = datetime.datetime(2020, 10, 5) # don't use data before this data to analyze disparities / demographics
POSITIVE_CLASSIFICATION_THRESHOLD = 0.77 # threshold to define a positive prediction
LOCATION_COL_TO_GROUP_ON = 'NAME' # This should be the name of the column we're analyzing location grouping at - e.g., corresponding to Census Block Group or Census tract.
TOTAL_POPULATION_COL = 'Estimate_Total' # needs to match whether using Census tract or Block group. 
WHITE_POPULATION_COL = 'Estimate_Total_Not_Hispanic_or_Latino_White_alone'
BLACK_POPULATION_COL = 'Estimate_Total_Not_Hispanic_or_Latino_Black_or_African_American_alone'
HISPANIC_POPULATION_COL = 'Estimate_Total_Hispanic_or_Latino'
ASIAN_POPULATION_COL = 'Estimate_Total_Not_Hispanic_or_Latino_Asian_alone'
POPULATION_COUNT_COLS = [WHITE_POPULATION_COL, BLACK_POPULATION_COL, HISPANIC_POPULATION_COL, ASIAN_POPULATION_COL, TOTAL_POPULATION_COL]
TIME_AND_DATE_COL = 'time_and_date_of_image'
DEMOGRAPHIC_COLS = ['density_cbg', # things we want to look at correlations with. Demographic cols may not be best name. 
                    'black_frac',
                    'white_frac', 
                    'distance_from_nearest_crime_6hr',
                    'distance_from_nearest_police_station',
                    'median_household_income']
PREDICTION_COLS = ['above_threshold', 'calibrated_prediction', 'prediction_adjusted_for_police_station_distance'] # columns with police car predictions. We define these
MIN_POPULATION_IN_AREA = 500
BOROUGH_COL = 'boroname'
NEIGHBORHOOD_COL = 'ntaname'
N_BOOTSTRAPS = 20
ZONE_THRESHOLD = 0.5


# read in data

In [4]:
d = []
for i in range(N_CHUNKS):
    d_i = pd.read_csv(BASE_CHUNKS_PATH % i)
    print('Read in chunk %i with %i rows' % (i, len(d_i)))
    d.append(d_i)
d = pd.concat(d)
#d.iloc[0][[a for a in d.columns if 'Margin of Error' not in a and 'Two races' not in a]] # just print out what dataframe looks like


Read in chunk 0 with 1115281 rows
Read in chunk 1 with 1115281 rows
Read in chunk 2 with 1115281 rows
Read in chunk 3 with 1115281 rows
Read in chunk 4 with 1115281 rows
Read in chunk 5 with 1115281 rows
Read in chunk 6 with 1115281 rows
Read in chunk 7 with 1115281 rows
Read in chunk 8 with 1115281 rows
Read in chunk 9 with 1115281 rows
Read in chunk 10 with 1115281 rows
Read in chunk 11 with 1115281 rows
Read in chunk 12 with 1115281 rows
Read in chunk 13 with 1115281 rows
Read in chunk 14 with 1115280 rows
Read in chunk 15 with 1115280 rows
Read in chunk 16 with 1115280 rows
Read in chunk 17 with 1115280 rows
Read in chunk 18 with 1115280 rows
Read in chunk 19 with 1115280 rows


# apply filters

In [5]:
# remove duplicates. 
duplicate_idxs = d.duplicated(subset=COLS_TO_DEDUPLICATE_ON)
print("warning: %i duplicates identified using %s, fraction %2.6f of rows; dropping rows" % (duplicate_idxs.sum(), COLS_TO_DEDUPLICATE_ON, duplicate_idxs.mean()))
d = d.loc[~duplicate_idxs].copy()

cbg_zone_data = pd.read_csv('/share/pierson/nexar_data/5_other_datasets/cbgs_zone_data.csv')
assert (1.*(cbg_zone_data['C'] > ZONE_THRESHOLD) + 1.*(cbg_zone_data['M'] > ZONE_THRESHOLD) + 1.*(cbg_zone_data['R'] > ZONE_THRESHOLD)).max() == 1
cbg_zone_dict = {}
for zone_val in ['C', 'M', 'R']:
    zones = cbg_zone_data.loc[cbg_zone_data[zone_val] >= ZONE_THRESHOLD]
    print("%i CBGs classified as %s" % (len(zones), zone_val))
    cbg_zone_dict.update(dict(zip(zones['GEOID20'].values, [zone_val for _ in range(len(zones))])))
print(len(cbg_zone_dict))
d['zone'] = d['GEOID20'].map(lambda x:cbg_zone_dict[x] if x in cbg_zone_dict else None)
print("zone classification of images")
print(d['zone'].value_counts(dropna=False))

def household_income_map(x):
    if x == '-':
        return None
    elif x == '250,000+':
        return 250000
    elif x == '2,500-':
        return 2500
    return float(x)

# define Census variables
d['median_household_income'] = d['median_household_income'].map(household_income_map)
d['white_frac'] = d[WHITE_POPULATION_COL] / d[TOTAL_POPULATION_COL]
d['black_frac'] = d[BLACK_POPULATION_COL] / d[TOTAL_POPULATION_COL]
assert d['white_frac'].dropna().max() <= 1
assert d['white_frac'].dropna().min() >= 0
assert d['black_frac'].dropna().max() <= 1
assert d['black_frac'].dropna().min() >= 0


# define time variables
d['date'] = d[TIME_AND_DATE_COL].map(lambda x:datetime.datetime.strptime(x.split()[0], '%Y-%m-%d'))
locations_by_date = d.groupby('date')[LOCATION_COL_TO_GROUP_ON].nunique()
print('unique locations by', locations_by_date)

# filter for dates with full coverage. 
print("In demographic analysis, filtering for locations after %s because more geographically representative" % MIN_DATE_FOR_DEMOGRAPHIC_ANALYSIS)
d_for_demo_analysis = d.loc[d['date'] >= MIN_DATE_FOR_DEMOGRAPHIC_ANALYSIS].copy()
print("%i/%i rows remaining" % (len(d_for_demo_analysis), len(d)))

for col in [WHITE_POPULATION_COL, BLACK_POPULATION_COL, HISPANIC_POPULATION_COL, ASIAN_POPULATION_COL, TOTAL_POPULATION_COL]:
    print("Setting fraction %2.6f of rows with %s = NA to 0" % (d_for_demo_analysis[col].isnull().mean(), 
                                                            col))
    d_for_demo_analysis.loc[d_for_demo_analysis[col].isnull(), col] = 0



383 CBGs classified as C
276 CBGs classified as M
5907 CBGs classified as R
6566
zone classification of images
R      15737220
C       3421338
M       2542381
NaN      604472
Name: zone, dtype: int64
unique locations by date
2020-03-05    1590
2020-03-06    1441
2020-03-13    1511
2020-03-20    1221
2020-03-27     728
2020-04-03     497
2020-04-10     555
2020-04-17     561
2020-04-24     875
2020-05-01     639
2020-05-08     735
2020-05-15     854
2020-05-22     956
2020-05-29     924
2020-06-05     662
2020-06-12     625
2020-06-19    1106
2020-06-26    1094
2020-07-03    1142
2020-07-26     128
2020-07-27     299
2020-07-31    1290
2020-08-01     326
2020-08-02     268
2020-08-03     359
2020-08-07    1384
2020-08-08     476
2020-08-09     460
2020-08-10     298
2020-08-14    1334
2020-08-15     538
2020-08-16     409
2020-08-17     405
2020-08-21    1311
2020-08-22     718
2020-08-23     746
2020-08-24     516
2020-08-28    1377
2020-08-29     722
2020-08-30     590
2020-08-31     

# compute probability measures using validation set

In [6]:
def calibrate_probabilities_using_valset(val_d, d_to_add_prediction_columns_to):
    """
    Annotate a dataframe, d_to_add_prediction_columns_to, with three prediction columns
    derived from the val set val_d.
    
    1. A simple binary variable with whether conf > POSITIVE_CLASSIFICATION_THRESHOLD
    2. A probabilistic prediction from val set: if above threshold, Pr(ground truth positive | above threshold in val set)
    and if below threshold, Pr(ground truth negative | below threshold in val set)
    3. A probability adjusted for police station distance. Not sure if this is a good thing to use, and should definitely check it is calibrated on test set if we do.
    """
    
    # 1. annotate with simple binary score
    assert val_d['Model_predicted_score'].isnull().sum() == 0
    val_d['classified_positive'] = val_d['Model_predicted_score'] > POSITIVE_CLASSIFICATION_THRESHOLD
    d_to_add_prediction_columns_to['above_threshold'] = (d_to_add_prediction_columns_to['conf'] > POSITIVE_CLASSIFICATION_THRESHOLD) * 1.
    
    # 2. compute probabilities given above/below threshold from val set
    p_positive_given_classified_positive = val_d.loc[val_d['classified_positive'] == True, 'ground_truth'].mean()
    p_positive_given_classified_negative = val_d.loc[val_d['classified_positive'] == False, 'ground_truth'].mean()
    print("Fraction of val set classified positive: %2.3f (%i rows)" % 
          (val_d['classified_positive'].mean(), val_d['classified_positive'].sum()))
    print("Pr(true positive | classified positive): %2.3f" % p_positive_given_classified_positive)
    print("Pr(true positive | classified negative): %2.3f" % p_positive_given_classified_negative)
    d_to_add_prediction_columns_to['calibrated_prediction'] = d_to_add_prediction_columns_to['above_threshold'].map(lambda x:p_positive_given_classified_positive if x == 1 else p_positive_given_classified_negative) 
    
    # 3. compute adjusted probability given police station distance. Not sure if this is necessary or wise, but adding just in case. 
    police_station_distance_model = sm.Logit.from_formula('ground_truth ~ Model_predicted_score + distance_from_nearest_police_station', data=val_d).fit()
    print(police_station_distance_model.summary())
    d_to_add_prediction_columns_to['Model_predicted_score'] = 0 # compute police-distance adjusted probability on d_to_add_prediction_columns_to. 
    d_to_add_prediction_columns_to.loc[~pd.isnull(d_to_add_prediction_columns_to['conf']), 'Model_predicted_score'] = d_to_add_prediction_columns_to['conf'].loc[~pd.isnull(d_to_add_prediction_columns_to['conf'])]
    assert d_to_add_prediction_columns_to['Model_predicted_score'].isnull().sum() == 0
    d_to_add_prediction_columns_to['prediction_adjusted_for_police_station_distance'] = police_station_distance_model.predict(d_to_add_prediction_columns_to).values
    del d_to_add_prediction_columns_to['Model_predicted_score']
    
    added_cols = ['above_threshold', 'calibrated_prediction', 'prediction_adjusted_for_police_station_distance']
    assert pd.isnull(d_to_add_prediction_columns_to[added_cols]).values.sum() == 0
    assert (d_to_add_prediction_columns_to[added_cols].values < 0).sum() == 0
    assert (d_to_add_prediction_columns_to[added_cols].values > 1).sum() == 0
    for col in added_cols:
        print("Mean value of prediction column %s: %2.3f; std %2.3f; > 0 %2.3f" % (
            col,
            d_to_add_prediction_columns_to[col].mean(), 
            d_to_add_prediction_columns_to[col].std(), 
            (d_to_add_prediction_columns_to[col] > 0).mean()))
    
    return d_to_add_prediction_columns_to
    
val_d = pd.read_csv(VALSET_PATH)
d_for_demo_analysis = calibrate_probabilities_using_valset(val_d=val_d, d_to_add_prediction_columns_to=d_for_demo_analysis)

Fraction of val set classified positive: 0.013 (265 rows)
Pr(true positive | classified positive): 0.777
Pr(true positive | classified negative): 0.002
Optimization terminated successfully.
         Current function value: 0.017459
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:           ground_truth   No. Observations:                20000
Model:                          Logit   Df Residuals:                    19997
Method:                           MLE   Df Model:                            2
Date:                Sun, 05 Feb 2023   Pseudo R-squ.:                  0.7376
Time:                        15:15:46   Log-Likelihood:                -349.18
converged:                       True   LL-Null:                       -1330.8
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------

# disparities analysis

In [7]:
# group by Census area. 
grouped_d = d_for_demo_analysis.groupby(LOCATION_COL_TO_GROUP_ON)[POPULATION_COUNT_COLS + 
                                                              DEMOGRAPHIC_COLS + 
                                                              PREDICTION_COLS].agg('mean')
for col in POPULATION_COUNT_COLS + DEMOGRAPHIC_COLS: # check consistent values by location for demographics. Should only be one value of population count per Census area, for example. 
    if col in ['distance_from_nearest_crime_6hr', 'distance_from_nearest_police_station']:
        continue
    assert d_for_demo_analysis.groupby(LOCATION_COL_TO_GROUP_ON)[col].nunique().map(lambda x:x in [0, 1]).all()

print("%i unique Census areas using column %s" % (len(grouped_d), LOCATION_COL_TO_GROUP_ON))
print("Population statistics by area")
print(grouped_d[TOTAL_POPULATION_COL].describe([0.01, 0.05, 0.1, 0.5, 0.9, 0.99]))
print("excluding census areas with population < %i keeps fraction %2.3f of population" % 
      (MIN_POPULATION_IN_AREA, 
       grouped_d.loc[grouped_d[TOTAL_POPULATION_COL] >= MIN_POPULATION_IN_AREA, TOTAL_POPULATION_COL].sum()/grouped_d[TOTAL_POPULATION_COL].sum()))
for col in POPULATION_COUNT_COLS: # sanity check that total counts look right. 
    print("summed values of %s: %i" % (col, grouped_d[col].sum()))
    

6602 unique Census areas using column NAME
Population statistics by area
count    6602.000000
mean     1269.244471
std       641.995903
min         0.000000
1%          0.000000
5%        357.000000
10%       579.000000
50%      1192.000000
90%      2090.000000
99%      3104.960000
max      8541.000000
Name: Estimate_Total, dtype: float64
excluding census areas with population < 500 keeps fraction 0.989 of population
summed values of Estimate_Total_Not_Hispanic_or_Latino_White_alone: 2676732
summed values of Estimate_Total_Not_Hispanic_or_Latino_Black_or_African_American_alone: 1795055
summed values of Estimate_Total_Hispanic_or_Latino: 2423869
summed values of Estimate_Total_Not_Hispanic_or_Latino_Asian_alone: 1185423
summed values of Estimate_Total: 8379552


In [8]:
def weighted_disparities_estimator(df, census_area_col, weighting_cols, total_population_col, estimate_col, check_consistent_vals_by_group):
    """
    Given a census dataframe
    group by census_area_col and compute the mean value of estimate_col in each census area
    then return weighted means across Census areas, 
    weighting each Census area by all the columns in weighting_cols and total_population_col. 
    We use this for computations of race-specific results. 
    Emma reviewed. 
    """
    grouped_d = df.groupby(census_area_col)[weighting_cols + [total_population_col, estimate_col]].mean().reset_index()
    if check_consistent_vals_by_group: # sanity check to make sure that values are consistent
        consistency_df = df.groupby(census_area_col)[weighting_cols + [total_population_col]].nunique()
        assert (consistency_df.values == 1).all()
        assert df[weighting_cols + [total_population_col, estimate_col]].isnull().values.sum() == 0
    results = {}
    for col in weighting_cols + [total_population_col]:
        results['%s_weighted_mean' % col] = (grouped_d[estimate_col] * grouped_d[col]).sum()/grouped_d[col].sum()
    for col in weighting_cols:
        results['%s_relative_to_average' % col] = results['%s_weighted_mean' % col]/results['%s_weighted_mean' % total_population_col]
    return results

def weighted_disparities_estimator_two_level_grouping(df, census_area_col, high_level_group_col, total_population_col, estimate_col, check_consistent_vals_by_group):
    """
    This function is similar to that above, but is (hopefully) a faster way to compute 
    two-level groupings (e.g., we want to compute borough-specific numbers, and weight by Census tract population within borough). 
    high_level_group col specifies the column we want to compute disparities over (e.g. borough). 
    All other columns are as explained above. 
    Verified that this gives identical results to function above for borough, zone, etc. 
    """
    if check_consistent_vals_by_group: # sanity check to make sure that values are consistent
        consistency_df = df.groupby(census_area_col)[high_level_group_col].nunique()
        assert ((consistency_df.values == 1) | (consistency_df.values == 0)).all()
    results = {}
    # first compute overall mean. 
    overall_mean_grouping = df.groupby(census_area_col)[[total_population_col, estimate_col]].mean().reset_index()
    results['%s_weighted_mean' % total_population_col] = (overall_mean_grouping[estimate_col] * overall_mean_grouping[total_population_col]).sum()/overall_mean_grouping[total_population_col].sum()
    high_level_grouping = df.groupby(high_level_group_col)
    all_names = []
    for name, group_df in high_level_grouping:
        if group_df[total_population_col].sum() == 0:
            print("Skipping %s because total population is 0" % name)
            continue
        all_names.append(name)
        second_level_grouping = group_df.groupby(census_area_col)[[total_population_col, estimate_col]].mean().reset_index()
        results['%s_weighted_mean' % name] = (second_level_grouping[estimate_col] * second_level_grouping[total_population_col]).sum()/second_level_grouping[total_population_col].sum()
    for name in all_names:
        results['%s_relative_to_average' % name] = results['%s_weighted_mean' % name]/results['%s_weighted_mean' % total_population_col]
    return results

def bootstrap_function_errorbars(df, fxn_to_apply, fxn_kwargs, n_bootstraps=100, filename=None):
    # compute the point estimate fxn_to_apply(df) on the original data
    # and then do bootstrap iterates. Emma reviewed. 
    bootstrap_statistics = []
    point_estimate = fxn_to_apply(df, check_consistent_vals_by_group=True, **fxn_kwargs)
    for bootstrap in tqdm(range(n_bootstraps)):
            bootstrap_df = df.sample(frac=1, replace=True)
            bootstrap_statistics.append(fxn_to_apply(bootstrap_df, check_consistent_vals_by_group=False, **fxn_kwargs))
    if filename is not None:
        with open(filename, 'w') as f:
            json.dump({'point_estimate':point_estimate, 'bootstrap_statistics':bootstrap_statistics}, f)
    return point_estimate, bootstrap_statistics

# print out a table. Emma reviewed. 
def create_table_from_bootstrap_results(bootstrap_point_estimate, bootstrap_statistics):
    bootstrap_statistics = pd.DataFrame(bootstrap_statistics)
    bootstrap_results_table = []
    for k in bootstrap_point_estimate.keys():
        lower_CI = np.percentile(bootstrap_statistics[k], 2.5)
        upper_CI = np.percentile(bootstrap_statistics[k], 97.5)
        bootstrap_results_table.append({'quantity':k, 
                                        'estimate':bootstrap_point_estimate[k], 
                                        #'bootstrap std':bootstrap_statistics[k].std(), 
                                        #'2.5% percentile':lower_CI, 
                                        #'97.5% percentile':upper_CI, 
                                        'percentile CI':'%2.3f (%2.3f, %2.3f)' % (bootstrap_point_estimate[k], 
                                                                                         lower_CI, 
                                                                                         upper_CI), 
                                       '1.96 sd CI':'%2.3f +/- %2.3f' % (bootstrap_point_estimate[k], 1.96 * bootstrap_statistics[k].std())})
    bootstrap_results_table = pd.DataFrame(bootstrap_results_table)
    return (bootstrap_results_table.loc[bootstrap_results_table['quantity'].map(lambda x:'relative_to_average' in x), 
                                       ['quantity', 'estimate', 'percentile CI', '1.96 sd CI']].sort_values(by='estimate')[::-1])


        
    

In [9]:
# race table (not conditioning on Zone). Emma reviewed. 
# compute point estimate and errorbars
bootstrap_point_estimate, bootstrap_statistics = bootstrap_function_errorbars(df=d_for_demo_analysis, 
                             fxn_to_apply=weighted_disparities_estimator, 
                             fxn_kwargs={'census_area_col':LOCATION_COL_TO_GROUP_ON, 
                                         'weighting_cols':[WHITE_POPULATION_COL, BLACK_POPULATION_COL, HISPANIC_POPULATION_COL, ASIAN_POPULATION_COL], 
                                         'total_population_col':TOTAL_POPULATION_COL, 
                                         'estimate_col':'calibrated_prediction'}, 
                             n_bootstraps=N_BOOTSTRAPS, 
                             filename='race_bootstraps.json')
create_table_from_bootstrap_results(bootstrap_point_estimate, bootstrap_statistics)




100%|███████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [44:48<00:00, 134.43s/it]


,quantity,estimate,percentile CI,1.96 sd CI
7,Estimate_Total_Hispanic_or_Latino_relative_to_...,1.049640,"1.050 (1.045, 1.055)",1.050 +/- 0.005
6,Estimate_Total_Not_Hispanic_or_Latino_Black_or...,1.040333,"1.040 (1.035, 1.045)",1.040 +/- 0.006
5,Estimate_Total_Not_Hispanic_or_Latino_White_al...,0.989513,"0.990 (0.985, 0.993)",0.990 +/- 0.005
8,Estimate_Total_Not_Hispanic_or_Latino_Asian_al...,0.878738,"0.879 (0.872, 0.883)",0.879 +/- 0.006


In [10]:
# race table CONDITIONING ON ZONE. Emma reviewed. 
# compute point estimate and errorbars
bootstrap_point_estimate, bootstrap_statistics = bootstrap_function_errorbars(
    df=d_for_demo_analysis.loc[d_for_demo_analysis['zone'] == 'R'], 
                             fxn_to_apply=weighted_disparities_estimator, 
                             fxn_kwargs={'census_area_col':LOCATION_COL_TO_GROUP_ON, 
                                         'weighting_cols':[WHITE_POPULATION_COL, BLACK_POPULATION_COL, HISPANIC_POPULATION_COL, ASIAN_POPULATION_COL], 
                                         'total_population_col':TOTAL_POPULATION_COL, 
                                         'estimate_col':'calibrated_prediction'}, 
                             n_bootstraps=N_BOOTSTRAPS, 
                             filename='race_residential_zones_only_bootstraps.json')
create_table_from_bootstrap_results(bootstrap_point_estimate, bootstrap_statistics)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [32:06<00:00, 96.31s/it]


,quantity,estimate,percentile CI,1.96 sd CI
6,Estimate_Total_Not_Hispanic_or_Latino_Black_or...,1.083814,"1.084 (1.080, 1.090)",1.084 +/- 0.006
7,Estimate_Total_Hispanic_or_Latino_relative_to_...,1.082602,"1.083 (1.077, 1.086)",1.083 +/- 0.005
5,Estimate_Total_Not_Hispanic_or_Latino_White_al...,0.953256,"0.953 (0.948, 0.957)",0.953 +/- 0.004
8,Estimate_Total_Not_Hispanic_or_Latino_Asian_al...,0.811473,"0.811 (0.806, 0.817)",0.811 +/- 0.007


In [9]:
# continuous variables: divide into quartile. Emma reviewed. 

# density_cbg, median_household_income

for col in ['median_household_income']: # ['density_cbg']
    percentile_cutoffs = [25, 50, 75]
    print("Fraction of missing values for %s: %2.6f" % (col, d_for_demo_analysis[col].isnull().mean()))
    d_for_col = d_for_demo_analysis.dropna(subset=[col]).copy()
    cutoff_vals = np.percentile(d_for_col[col], percentile_cutoffs)
    cutoff_vals = [-np.inf] + list(cutoff_vals) + [np.inf]
    print('cutoffs for %s' % col, cutoff_vals)
    d_for_col['%s_quartile' % col] = None

    for i in range(len(cutoff_vals) - 1):
        quartile_idxs = d_for_col[col].map(lambda x:(x >= cutoff_vals[i]) & (x < cutoff_vals[i + 1]))
        d_for_col.loc[quartile_idxs, '%s_quartile' % col] = '%s_quartile_%i' % (col, i + 1)
        print('number of rows in %s: %i' % ('%s_quartile' % col, quartile_idxs.sum()))
    bootstrap_point_estimate, bootstrap_statistics = bootstrap_function_errorbars(df=d_for_col, 
                             fxn_to_apply=weighted_disparities_estimator_two_level_grouping, 
                             fxn_kwargs={'census_area_col':LOCATION_COL_TO_GROUP_ON, 
                                         'high_level_group_col':'%s_quartile' % col,
                                         'total_population_col':TOTAL_POPULATION_COL, 
                                         'estimate_col':'calibrated_prediction'}, 
                             n_bootstraps=N_BOOTSTRAPS, 
                             filename='%s_bootstraps.json' % col)

    print(create_table_from_bootstrap_results(bootstrap_point_estimate, bootstrap_statistics))

Fraction of missing values for median_household_income: 0.174104
cutoffs for median_household_income [-inf, 51875.0, 77000.0, 115385.0, inf]
number of rows in median_household_income_quartile: 3864047
number of rows in median_household_income_quartile: 3864508
number of rows in median_household_income_quartile: 3865710
number of rows in median_household_income_quartile: 3866355


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [54:43<00:00, 164.17s/it]


                                            quantity  estimate  \
8  median_household_income_quartile_4_relative_to...  1.156263   
5  median_household_income_quartile_1_relative_to...  1.122250   
7  median_household_income_quartile_3_relative_to...  0.905192   
6  median_household_income_quartile_2_relative_to...  0.881983   

          percentile CI       1.96 sd CI  
8  1.156 (1.145, 1.165)  1.156 +/- 0.011  
5  1.122 (1.110, 1.132)  1.122 +/- 0.012  
7  0.905 (0.899, 0.915)  0.905 +/- 0.010  
6  0.882 (0.875, 0.888)  0.882 +/- 0.009  


In [10]:
# zone table. Emma reviewed. 
bootstrap_point_estimate, bootstrap_statistics = bootstrap_function_errorbars(df=d_for_demo_analysis, 
                             fxn_to_apply=weighted_disparities_estimator_two_level_grouping, 
                             fxn_kwargs={'census_area_col':LOCATION_COL_TO_GROUP_ON, 
                                         'high_level_group_col':'zone',
                                         'total_population_col':TOTAL_POPULATION_COL, 
                                         'estimate_col':'calibrated_prediction'}, 
                             n_bootstraps=N_BOOTSTRAPS, 
                             filename='zone_bootstraps.json')

create_table_from_bootstrap_results(bootstrap_point_estimate, bootstrap_statistics)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [1:02:14<00:00, 186.75s/it]


,quantity,estimate,percentile CI,1.96 sd CI
4,C_relative_to_average,1.985214,"1.985 (1.949, 2.005)",1.985 +/- 0.033
5,M_relative_to_average,0.948387,"0.948 (0.930, 0.968)",0.948 +/- 0.019
6,R_relative_to_average,0.937805,"0.938 (0.937, 0.940)",0.938 +/- 0.002


In [11]:
# boro table. Emma reviewed. 
bootstrap_point_estimate, bootstrap_statistics = bootstrap_function_errorbars(df=d_for_demo_analysis, 
                             fxn_to_apply=weighted_disparities_estimator_two_level_grouping, 
                             fxn_kwargs={'census_area_col':LOCATION_COL_TO_GROUP_ON, 
                                         'high_level_group_col':'boroname',
                                         'total_population_col':TOTAL_POPULATION_COL, 
                                         'estimate_col':'calibrated_prediction'}, 
                             n_bootstraps=N_BOOTSTRAPS, 
                             filename='boro_bootstraps.json')

create_table_from_bootstrap_results(bootstrap_point_estimate, bootstrap_statistics)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [1:04:47<00:00, 194.38s/it]


,quantity,estimate,percentile CI,1.96 sd CI
8,Manhattan_relative_to_average,1.619059,"1.619 (1.610, 1.625)",1.619 +/- 0.008
6,Bronx_relative_to_average,0.994406,"0.994 (0.983, 1.007)",0.994 +/- 0.014
7,Brooklyn_relative_to_average,0.932224,"0.932 (0.926, 0.937)",0.932 +/- 0.007
9,Queens_relative_to_average,0.738071,"0.738 (0.732, 0.744)",0.738 +/- 0.007
10,Staten Island_relative_to_average,0.507765,"0.508 (0.495, 0.521)",0.508 +/- 0.016


In [12]:
# neighborhood table. Emma reviewed. 
bootstrap_point_estimate, bootstrap_statistics = bootstrap_function_errorbars(df=d_for_demo_analysis, 
                             fxn_to_apply=weighted_disparities_estimator_two_level_grouping, 
                             fxn_kwargs={'census_area_col':LOCATION_COL_TO_GROUP_ON, 
                                         'high_level_group_col':NEIGHBORHOOD_COL,
                                         'total_population_col':TOTAL_POPULATION_COL, 
                                         'estimate_col':'calibrated_prediction'}, 
                             n_bootstraps=N_BOOTSTRAPS, 
                             filename='neighborhood_bootstraps.json')

create_table_from_bootstrap_results(bootstrap_point_estimate, bootstrap_statistics)

Skipping Astoria Park because total population is 0
Skipping Brooklyn Navy Yard because total population is 0
Skipping Calvary & Mount Zion Cemeteries because total population is 0
Skipping Canarsie Park & Pier because total population is 0
Skipping Claremont Park because total population is 0
Skipping Crotona Park because total population is 0
Skipping Cunningham Park because total population is 0
Skipping Ferry Point Park-St. Raymond Cemetery because total population is 0
Skipping Flushing Meadows-Corona Park because total population is 0
Skipping Forest Park because total population is 0
Skipping Fort Totten because total population is 0
Skipping Freshkills Park (North) because total population is 0
Skipping Great Kills Park because total population is 0
Skipping Green-Wood Cemetery because total population is 0
Skipping Highbridge Park because total population is 0
Skipping Highland Park-Cypress Hills Cemeteries (South) because total population is 0
Skipping John F. Kennedy Interna


  0%|                                                                                                                                        | 0/20 [00:00<?, ?it/s]

Skipping Astoria Park because total population is 0
Skipping Brooklyn Navy Yard because total population is 0
Skipping Calvary & Mount Zion Cemeteries because total population is 0
Skipping Canarsie Park & Pier because total population is 0
Skipping Claremont Park because total population is 0
Skipping Crotona Park because total population is 0
Skipping Cunningham Park because total population is 0
Skipping Ferry Point Park-St. Raymond Cemetery because total population is 0
Skipping Flushing Meadows-Corona Park because total population is 0
Skipping Forest Park because total population is 0
Skipping Fort Totten because total population is 0
Skipping Freshkills Park (North) because total population is 0
Skipping Great Kills Park because total population is 0
Skipping Green-Wood Cemetery because total population is 0
Skipping Highbridge Park because total population is 0
Skipping Highland Park-Cypress Hills Cemeteries (South) because total population is 0
Skipping John F. Kennedy Interna


  5%|██████▎                                                                                                                      | 1/20 [04:15<1:20:50, 255.30s/it]

Skipping Astoria Park because total population is 0
Skipping Brooklyn Navy Yard because total population is 0
Skipping Calvary & Mount Zion Cemeteries because total population is 0
Skipping Canarsie Park & Pier because total population is 0
Skipping Claremont Park because total population is 0
Skipping Crotona Park because total population is 0
Skipping Cunningham Park because total population is 0
Skipping Ferry Point Park-St. Raymond Cemetery because total population is 0
Skipping Flushing Meadows-Corona Park because total population is 0
Skipping Forest Park because total population is 0
Skipping Fort Totten because total population is 0
Skipping Freshkills Park (North) because total population is 0
Skipping Great Kills Park because total population is 0
Skipping Green-Wood Cemetery because total population is 0
Skipping Highbridge Park because total population is 0
Skipping Highland Park-Cypress Hills Cemeteries (South) because total population is 0
Skipping John F. Kennedy Interna

 10%|████████████▌                                                                                                                | 2/20 [10:52<1:37:50, 326.12s/it]


KeyboardInterrupt: 

# Estimator described in Google doc applied to estimate disparities

In [ ]:
for prediction_col in PREDICTION_COLS:
    print("Using prediction col", prediction_col)
    estimates = {}
    for demo_col in POPULATION_COUNT_COLS:
        if demo_col == TOTAL_POPULATION_COL:
            continue
        # compute weighted mean as described in Census tract. 
        grouped_mean = (grouped_d[prediction_col] * grouped_d[demo_col]).sum()/grouped_d[demo_col].sum()
        print(demo_col, grouped_mean)
        estimates[demo_col] = grouped_mean
    print("Ratio of Black estimate to white estimate: %2.3f" % (estimates[BLACK_POPULATION_COL]/estimates[WHITE_POPULATION_COL]))

# scatterplots

Basically these plot Pr(police car) by Census area against various demographic variables. 

In [ ]:
for var in DEMOGRAPHIC_COLS:
    for prediction_col in PREDICTION_COLS:
        plt.figure(figsize=[4, 4])
        # in making these scatterplots, we drop very small Census areas (smaller than MIN_POPULATION_IN_AREA)
        # just to avoid very noisy estimates. 
        # The weighted correlation measure verifies that this doesn't change results a lot. 
        d_to_plot = grouped_d.loc[grouped_d[TOTAL_POPULATION_COL] > MIN_POPULATION_IN_AREA].dropna(subset=[var, prediction_col])
        plt.scatter(d_to_plot[var], 
                    d_to_plot[prediction_col], 
                    s=d_to_plot[TOTAL_POPULATION_COL] * 1e-3)
        plt.xlabel(var, fontsize=16)
        plt.ylabel(prediction_col, fontsize=16)
        pearson_r, pearson_p = pearsonr(d_to_plot[var], d_to_plot[prediction_col])
        spearman_r, spearman_p = spearmanr(d_to_plot[var], d_to_plot[prediction_col])
        d_for_weighted_correlation = grouped_d.dropna(subset=[var, prediction_col])
        weighted_correlation = DescrStatsW(d_for_weighted_correlation[[var, prediction_col]], 
                                           weights=d_for_weighted_correlation[TOTAL_POPULATION_COL]).corrcoef[0][1]
        plt.title('pearsonr %2.3f, p=%2.3e (population-weighted %2.3f)\nspearmanr %2.3f, p=%2.3e' % (pearson_r, pearson_p, weighted_correlation, spearman_r, spearman_p))
        plt.ylim([0, 0.05])
        plt.show()

# correlations between all measures

In [ ]:
grouped_d.loc[grouped_d[TOTAL_POPULATION_COL] > MIN_POPULATION_IN_AREA, DEMOGRAPHIC_COLS].corr(method='pearson')

In [ ]:
grouped_d.loc[grouped_d[TOTAL_POPULATION_COL] > MIN_POPULATION_IN_AREA, DEMOGRAPHIC_COLS].corr(method='spearman')

# broken down by neighborhood

In [ ]:
for col in PREDICTION_COLS:
    print("\n\nneighborhoods with highest mean values of %s" % col)
    print(d_for_demo_analysis
          .groupby(NEIGHBORHOOD_COL)[col]
          .agg(['mean', 'size'])
          .reset_index()
          .sort_values(by='mean')[::-1]
          .head(n=25))

In [ ]:
for col in PREDICTION_COLS:
    print("\n\nneighborhoods with highest mean values of %s" % col)
    print(d_for_demo_analysis
          .groupby(BOROUGH_COL)[col]
          .agg(['mean', 'size'])
          .reset_index().sort_values(by='mean')[::-1]
          .head(n=50))